In [3]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored

In [18]:
# Load data
print(colored("Loading train and test data", "green"))
train_data = pd.read_csv('twt_#ifc.csv')
test_data = pd.read_csv('f1.csv')
print(colored("Data loaded", "green"))


Loading train and test data
Data loaded


In [19]:
train_data.shape

(1749, 8)

In [20]:
test_data.shape

(508, 8)

In [21]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(train_data['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(train_data['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(test_data['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [22]:
model_prediction = Sequential()
model_prediction.add(Embedding(10000, 50, input_length=34))
model_prediction.add(Dropout(0.5))
model_prediction.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))
model_prediction.add(GlobalMaxPooling1D())
model_prediction.add(Dense(250))
model_prediction.add(Activation('relu'))
model_prediction.add(Dense(3))
model_prediction.add(Activation('sigmoid'))
model_prediction.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   

model_prediction.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 34, 50)            500000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 34, 50)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 32, 250)           37750     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 250)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               62750     
_________________________________________________________________
activation_9 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                

In [23]:

# Training the model
print(colored("Training the LSTM model", "green"))
history = model_prediction.fit(train_tweets, pd.get_dummies(train_data['sent_score']).values, epochs = 10, batch_size = 34)
print(colored(history, "green"))

Training the LSTM model


C:\Users\TEMP.DESKTOP-8R6RI8B.000\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1749/1749 [==============================] - 3s 2ms/step - loss: 0.5771 - accuracy: 0.6930
Epoch 2/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0.5351 - accuracy: 0.7208
Epoch 3/10
1749/1749 [==============================] - 2s 994us/step - loss: 0.4508 - accuracy: 0.7669
Epoch 4/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0.3710 - accuracy: 0.8189
Epoch 5/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0.2963 - accuracy: 0.8788
Epoch 6/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0.2235 - accuracy: 0.9156
Epoch 7/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0.1695 - accuracy: 0.9375
Epoch 8/10
1749/1749 [==============================] - 2s 996us/step - loss: 0.1311 - accuracy: 0.9562
Epoch 9/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0.0994 - accuracy: 0.9651
Epoch 10/10
1749/1749 [==============================] - 2s 1ms/step - loss: 0

In [24]:

# Testing the model
print(colored("Testing the LSTM model", "green"))
score, accuracy = model_prediction.evaluate(test_tweets, pd.get_dummies(test_data['sent_score']).values, batch_size = 34)
print("Test accuracy: {}".format(accuracy))
print("Test loss: {}".format(1-accuracy))

Testing the LSTM model
508/508 [==============================] - 0s 616us/step
Test accuracy: 0.7237532734870911
Test loss: 0.27624672651290894
